<!-- <a href="https://colab.research.google.com/github/demystify-systems/edge-channel-suite/blob/main/demo/HelloSlackDocker2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> -->

In [ ]:
# # Parameters
# saas_edge_id = "60ff8220-4d6f-4433-afd4-67952070e964"
# slack_webhook_url = "os.getenv("SLACK_WEBHOOK_URL", "")"
# job_id = "a8ddf474-8fbe-4f0d-80a9-fc1fd80d103e"

In [ ]:
!pip install pg8000 sqlalchemy pandas requests openpyxl zipfile36 psycopg2-binary --quiet
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib google-cloud-storage --quiet

import pg8000
import json
import os
import traceback
from datetime import datetime,date
import requests
from uuid import UUID
from google.cloud import storage
from google.oauth2 import service_account
from concurrent.futures import ThreadPoolExecutor, as_completed
import mimetypes

import pytz
from jsonschema import validate, ValidationError, SchemaError
import sys


import psycopg2
from psycopg2.extras import RealDictCursor
import json
from psycopg2 import sql as psql 
from typing import List, Dict,Tuple, Any, Optional,Set,Any
from datetime import datetime
from decimal import Decimal
import zipfile
import requests
import pandas as pd
from traceback import print_exc
from openpyxl import load_workbook
from io import BytesIO
from slack_sdk import WebhookClient
# global_message= f"{customer_id} -- is executing the job from google cloud runner with DB access"

In [ ]:
# import importlib

# # Step 1: Build sdk path from current location
# root_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
# sdk_path = os.path.join(root_path, 'sdk')

# # Step 2: Clean sys.path and insert sdk_path
# sys.path = [p for p in sys.path if 'sdk' not in p]
# if sdk_path not in sys.path:
#     sys.path.insert(0, sdk_path)

# print(f"SDK path set to: {sdk_path}")

# # Step 3: Import and force reload the module
# import export_and_import_transformation
# importlib.reload(export_and_import_transformation)

# # Step 4: Now access your function
# bulk_apply_pipe_rules = export_and_import_transformation.bulk_apply_pipe_rules

In [ ]:
db_user = os.getenv("DB_USER", "postgres")                # Default to 'postgres' if env variable not set
db_pass = os.getenv("DB_PASSWORD", "saasdbforwindmill2023")
db_name = os.getenv("DB_NAME", "catalog-edge-db")
db_host = "/cloudsql/saastify-base-wm:us-central1:saastify-pgdb-us"
# db_host = "127.0.0.1"
db_port = 5432

## SDK Methods

In [ ]:
# Add this after the database configuration section (in[3])

def get_db_connection():
    """
    Creates and returns a database connection using global configuration.
    Handles both Unix socket and TCP connections.
    """
    try:
        # Determine if we're using Unix socket or TCP
        if db_host.startswith('/cloudsql/'):
            connection = pg8000.connect(
                user=db_user,
                password=db_pass,
                database=db_name,
                unix_sock=db_host + "/.s.PGSQL.5432"
            )
        else:
            connection = pg8000.connect(
                user=db_user,
                password=db_pass,
                database=db_name,
                host=db_host,
                port=db_port
            )
        return connection
    except Exception as e:
        print("ERROR: Failed to establish database connection")
        print(f"ERROR: {str(e)}")
        print(f"ERROR: {traceback.format_exc()}")
        raise

In [ ]:

def download_url(url, file_name=""):
    get_response = requests.get(url, stream=True)
    if file_name == "":
        file_name = url.split("/")[-1]
    with open(file_name, "wb") as f:
        for chunk in get_response.iter_content(chunk_size=1024):
            if chunk:  # filter out keep-alive new chunks
                f.write(chunk)

    return file_name

class FileParser(object):
    def load(self, url):
        self.url = url
        self.file_name = download_url(url)
        self.file_type = self.file_name.split(".")[-1]
        print("The URL file type is : ", self.file_type)
        method_name = "parse_" + self.file_type
        method = getattr(self, method_name, lambda: "Invalid")
        return method()

    def infer_schema(self):
        self.df.info()
        self.columns = list(self.df.columns.values.tolist())
        print("List of all columns are : ", self.columns)
        print("##### Pandas inferred Schema")
        pandas_schema = self.df.columns.to_series().groupby(self.df.dtypes).groups
        print(pandas_schema)

    def parse_xlsx(self):
        return self.parse_excel()

    def parse_xlsm(self):
        return self.parse_excel()

    def parse_xls(self):
        return self.parse_excel()

    def parse_csv(self):
        df = pd.read_csv(self.file_name, sep=",", header=0)
        return df

    def parse_zip(self):
        with zipfile.ZipFile(self.file_name, "r") as zip_ref:
            zip_ref.extractall()
            extracted_files = zip_ref.namelist()
            print(extracted_files)
            return extracted_files

    def parse_tsv(self):
        df = pd.read_csv(self.file_name, sep="\t", header=0)
        return df

    def parse_json(self):
        df = pd.read_json(self.file_name)
        return df

    def parse_txt(self):
        df = pd.read_csv(self.file_name, sep=" ")
        return df

    def parse_excel(self):
        try:
            xls = pd.ExcelFile(self.file_name)
            # Read the first sheet by default
            sheet_name = xls.sheet_names[0]
            df = pd.read_excel(xls, sheet_name=sheet_name)
            return df
        except Exception as e:
            print(f"ERROR: Failed to parse Excel file: {str(e)}")
            print(f"ERROR: {traceback.format_exc()}")
            return pd.DataFrame()


In [ ]:
def send_slack_message(message, webhook_url):
    try:
        print("INFO: Attempting to send a message to Slack...")
        headers = {"Content-Type": "application/json"}
        payload = {"text": message}
        response = requests.post(webhook_url, data=json.dumps(payload), headers=headers)
        
        if response.status_code == 200:
            print("INFO: Slack message sent successfully.")
        else:
            print("WARNING: Failed to send Slack message.")
            print(f"WARNING: Status code: {response.status_code}")
            print(f"WARNING: Response: {response.text}")

    except Exception as e:
        print("ERROR: An error occurred while sending a Slack message.")
        print(f"ERROR: {e}")
        print(f"ERROR: {traceback.format_exc()}")

In [ ]:
# Database configuration - credentials should ideally be stored in environment variables for security


def fetch_saas_edge_jobs_details(job_id, saas_edge_id):
    try:
        print("INFO: Attempting to connect to Cloud SQL instance...")
        
        # Connect to the Cloud SQL instance
        conn = get_db_connection()
        print("INFO: Connected to Cloud SQL successfully.")

        # Define the query
        query = """
        SELECT * FROM public.saas_edge_jobs
        WHERE request_id = %s AND saas_edge_id = %s;
        """
        print(f"INFO: Executing query: {query} with job_id: {job_id} and saas_edge_id: {saas_edge_id}")

        # Execute the query
        cursor = conn.cursor()
        cursor.execute(query, (job_id, saas_edge_id))
        result = cursor.fetchall()

        if not result:
            print(f"WARNING: No results found for job_id: {job_id} and saas_edge_id: {saas_edge_id}")
        else:
            print(f"INFO: Query executed successfully. Number of records fetched: {len(result)}")

        # Process and convert datetime and UUID objects to strings for JSON serialization
        row_data = [
            {k: (v.isoformat() if isinstance(v, datetime) else str(v) if isinstance(v, UUID) else v) 
             for k, v in zip([col[0] for col in cursor.description], row)}
            for row in result
        ]
        
        print("INFO: Data processed for JSON serialization.")

        # Close the connection
        cursor.close()
        conn.close()
        print("INFO: Database connection closed.")
        
        return row_data

    except Exception as e:
        print("ERROR: An error occurred while connecting to Cloud SQL or executing the query.")
        print(f"ERROR: {e}")
        print(f"ERROR: {traceback.format_exc()}")  # Full stack trace for debugging
        return None


In [ ]:
def fetch_channel_attr_template(saas_edge_id, template_id):
    try:
        print("INFO: Attempting to connect to Cloud SQL instance...")
        
        conn = get_db_connection()
        print("INFO: Connected to Cloud SQL successfully.")

        query = """
        SELECT * FROM public.saas_channel_attr_template
        WHERE saas_edge_id = %s AND template_id = %s;
        """
        print(f"INFO: Executing query for saas_edge_id: {saas_edge_id} and template_id: {template_id}")

        cursor = conn.cursor()
        cursor.execute(query, (saas_edge_id, template_id))
        result = cursor.fetchall()

        if not result:
            print(f"WARNING: No results found for saas_edge_id: {saas_edge_id} and template_id: {template_id}")
            return None
            
        print(f"INFO: Query executed successfully. Number of records fetched: {len(result)}")

        # Process results
        row_data = [
            {k: (v.isoformat() if isinstance(v, datetime) else str(v) if isinstance(v, UUID) else v) 
             for k, v in zip([col[0] for col in cursor.description], row)}
            for row in result
        ]
        
        print("INFO: Data processed for JSON serialization.")

        cursor.close()
        conn.close()
        print("INFO: Database connection closed.")
        
        return row_data

    except Exception as e:
        print("ERROR: An error occurred while fetching channel attribute template data.")
        print(f"ERROR: {e}")
        print(f"ERROR: {traceback.format_exc()}")
        return None



In [ ]:
def fetch_saas_channel_templates(saas_edge_id, template_id):
    try:
        print("INFO: Attempting to connect to Cloud SQL instance...")
        
        conn = get_db_connection()
        print("INFO: Connected to Cloud SQL successfully.")

        query = """
        SELECT * FROM public.saas_channel_templates
        WHERE saas_edge_id = %s AND template_id = %s;
        """
        print(f"INFO: Executing query for saas_edge_id: {saas_edge_id} and template_id: {template_id}")

        cursor = conn.cursor()
        cursor.execute(query, (saas_edge_id, template_id))
        result = cursor.fetchall()

        if not result:
            print(f"WARNING: No results found for saas_edge_id: {saas_edge_id} and template_id: {template_id}")
            return None

        print(f"INFO: Query executed successfully. Number of records fetched: {len(result)}")

        # Process results and handle datetime/UUID serialization
        row_data = [
            {k: (v.isoformat() if isinstance(v, datetime) else str(v) if isinstance(v, UUID) else v) 
             for k, v in zip([col[0] for col in cursor.description], row)}
            for row in result
        ]
        
        print("INFO: Data processed for JSON serialization.")

        cursor.close()
        conn.close()
        print("INFO: Database connection closed.")
        
        return row_data[0] if row_data else None  # Return first row since template_id should be unique

    except Exception as e:
        print("ERROR: An error occurred while fetching channel template data.")
        print(f"ERROR: {e}")
        print(f"ERROR: {traceback.format_exc()}")
        return None

## Job Methods

In [ ]:

# Main execution code
try:
    job_id = job_id  # Replace with the actual job_id you want to query
    saas_edge_id = saas_edge_id  # Replace with the actual saas_edge_id you want to query
    slack_url = os.getenv("SLACK_WEBHOOK_URL", slack_webhook_url)
    
    if not slack_url:
        raise ValueError("Slack webhook URL not set in environment variables")

    # Fetch data from the database
    print(f"INFO: Fetching details for job_id: {job_id} and saas_edge_id: {saas_edge_id}")
    row_data = fetch_saas_edge_jobs_details(job_id, saas_edge_id)

    if not row_data:
        print("WARNING: No data retrieved or an error occurred during the fetch operation.")

except Exception as main_e:
    print("ERROR: An unexpected error occurred in the main execution block.")
    print(f"ERROR: {main_e}")
    print(f"ERROR: {traceback.format_exc()}")  # Full stack trace for debugging


request_args = row_data[0].get("request_args",{})

print(request_args)

In [ ]:
row_data

In [ ]:
job_path = request_args.get("job_path", "")
print("job_pathh",job_path)

In [ ]:
# Function to get the database configuration
db_config = {
    # "host":"localhost",
    "host": "/cloudsql/saastify-base-wm:us-central1:saastify-pgdb-us",
    "port": 5432,
    "database": "catalog-edge-db",
    "user": "postgres",
    "password": "saasdbforwindmill2023"
}


# Function to construct the PostgreSQL connection string
def create_connection_string() -> str:
    try:
        connection_string = (
            f"dbname={db_config['database']} "
            f"user={db_config['user']} "
            f"password={db_config['password']} "
            f"host={db_config['host']} "
            f"port={db_config['port']} "
        )
        return connection_string
    except KeyError as e:
        print(f"Missing key in DB configuration: {e}")
        raise
    except Exception as e:
        print(f"Error creating connection string: {e}")
        raise


# Helper function to convert datetime and decimal objects to serializable format
def custom_serializer(obj):
    if isinstance(obj, datetime):
        return obj.isoformat()
    elif isinstance(obj, Decimal):
        return float(obj)
    raise TypeError(f"Type {obj.__class__.__name__} not serializable")


# Function to fetch template attributes
def fetch_template_attributes(
    saas_edge_id: str, template_id: str
) -> List[Dict[str, Any]]:
    connection_string = create_connection_string()
    connection = None
    try:
        # Establish the connection
        connection = psycopg2.connect(connection_string, cursor_factory=RealDictCursor)
        cursor = connection.cursor()

        # Prepare the query to fetch attributes for the given saas_edge_id and template_id
        query = """
            SELECT * FROM saas_channel_attr_template 
            WHERE saas_edge_id = %s AND template_id = %s 
            ORDER BY column_pos ASC NULLS LAST, column_name ASC
        """
        print("Executing query:", query)

        # Execute the query with saas_edge_id and template_id as parameters
        cursor.execute(query, (saas_edge_id, template_id))
        response_data = cursor.fetchall()

        # If no data is fetched, handle gracefully
        if not response_data:
            print(
                f"No template attributes found for saas_edge_id: {saas_edge_id} and template_id: {template_id}"
            )
            return []

        print("Number of rows fetched:", len(response_data))
        return response_data

    except psycopg2.Error as e:
        print(f"Database error: {e}")
        raise
    except Exception as e:
        print(f"Error fetching template attributes: {e}")
        raise
    finally:
        if connection:
            cursor.close()
            connection.close()


# Function to generate JSON Schema dynamically, including hierarchy support
# Also generates the order array based on column_pos or alphabetic order


def generate_json_schema(template_attrs: List[Dict[str, Any]]) -> Dict[str, Any]:
    try:
        # Base schema
        schema = {
            "$schema": "https://json-schema.org/draft/2020-12/schema",
            "type": "object",
            "properties": {},
            "required": [],
            "order": [],
        }

        # Helper to handle hierarchical properties
        def add_property(
            properties: Dict[str, Any],
            attr: Dict[str, Any],
            parent_property: Optional[str] = None,
        ):
            column_name = attr["column_name"]
            data_type = (attr.get("data_type") or "text").lower()
            data_type_format = attr.get("data_type_format")

            # Map data_type to JSON schema types
            json_type_mapping = {
                "text": "string",
                "number": "number",
                "integer": "integer",
                "boolean": "boolean",
                "array": "array",
                "object": "object",
                "decimal": "number",
                "timestamp": "string",
                "url": "string",
                "date": "string",
                "datetime": "string",
                "time": "string",
            }
            field_type = json_type_mapping.get(data_type, "string")

            # Define the property schema
            property_schema = {
                "title": attr.get("custom_title", column_name),
                "type": field_type,
                "description": attr.get("description", f"Field for {column_name}"),
                "default": attr.get("default_value"),
                "nullable": attr.get("is_nullable", True),
            }

            # Handle data_type_format if available
            if data_type_format:
                property_schema["format"] = data_type_format

            # Handle multi-valued fields (array)
            if attr.get("accepts_multiple_values", False):
                property_schema = {
                    "type": "array",
                    "items": {"type": field_type},
                    "description": f"Array of {data_type} values for {column_name}",
                }

            # Add validation rules
            validation_rules = attr.get("validation_rules", {})
            if validation_rules:
                if "minLength" in validation_rules:
                    property_schema["minLength"] = validation_rules["minLength"]
                if "maxLength" in validation_rules:
                    property_schema["maxLength"] = validation_rules["maxLength"]
                if "pattern" in validation_rules:
                    property_schema["pattern"] = validation_rules["pattern"]
                if "enum" in validation_rules:
                    property_schema["enum"] = validation_rules["enum"]

            # Add to the appropriate place in the properties structure
            if parent_property:
                if "properties" not in properties[parent_property]:
                    properties[parent_property]["properties"] = {}
                properties[parent_property]["properties"][column_name] = property_schema
            else:
                properties[column_name] = property_schema

            # Add to required list if mandatory
            if attr.get("mandatory", False):
                if parent_property:
                    if "required" not in properties[parent_property]:
                        properties[parent_property]["required"] = []
                    properties[parent_property]["required"].append(column_name)
                else:
                    schema["required"].append(column_name)

            # Add to order list
            if not parent_property:
                schema["order"].append(column_name)

        # Process attributes considering hierarchy
        for attr in template_attrs:
            parent_id = attr.get("parent_id")
            if parent_id:
                # Find the parent property in existing attributes
                parent_attr = next(
                    (a for a in template_attrs if a["template_attr_id"] == parent_id),
                    None,
                )
                if parent_attr:
                    add_property(schema["properties"], attr, parent_attr["column_name"])
            else:
                add_property(schema["properties"], attr)

        return schema

    except Exception as e:
        print(f"Error generating JSON schema: {e}")
        raise


# Function to insert or update the generated schema in the database
def insert_or_update_schema(
    saas_edge_id: str, template_id: str, schema: Dict[str, Any]
) -> None:
    connection_string = create_connection_string()
    connection = None
    try:
        # Establish the connection
        connection = psycopg2.connect(connection_string)
        cursor = connection.cursor()

        # Convert schema dictionary to JSON string
        schema_json = json.dumps(schema)

        # Insert or update the schema using ON CONFLICT clause
        query = """
            INSERT INTO saas_template_schema (saas_edge_id, template_id, schema, created_at, updated_at)
            VALUES (%s, %s, %s, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP)
            ON CONFLICT (saas_edge_id, template_id)
            DO UPDATE SET
                schema = EXCLUDED.schema,
                updated_at = CURRENT_TIMESTAMP;
        """
        print("Executing query:", query)

        # Execute the query with provided parameters
        cursor.execute(query, (saas_edge_id, template_id, schema_json))
        connection.commit()
        print("Schema inserted or updated successfully.")

    except psycopg2.Error as e:
        print(f"Database error while inserting/updating schema: {e}")
        raise
    except Exception as e:
        print(f"Error in insert_or_update_schema function: {e}")
        raise
    finally:
        if connection:
            cursor.close()
            connection.close()


def get_existing_schema(saas_edge_id: str, template_id: str) -> Any:
    """
    Fetch the existing schema for the given saas_edge_id and template_id from the saas_template_schema table.
    """
    connection_string = create_connection_string()
    connection = None
    try:
        # Establish database connection
        connection = psycopg2.connect(connection_string, cursor_factory=RealDictCursor)
        cursor = connection.cursor()

        # Query to check for existing schema
        query = """
            SELECT schema FROM saas_template_schema
            WHERE saas_edge_id = %s AND template_id = %s
        """
        cursor.execute(query, (saas_edge_id, template_id))
        result = cursor.fetchone()

        if result:
            return result["schema"]
        return None
    except Exception as e:
        print(f"Error fetching schema: {e}")
        raise
    finally:
        if connection:
            cursor.close()
            connection.close()


def genarate_template_schema(saas_edge_id: str, template_id: str) -> Dict[str, Any]:
    """
    Main function to fetch or generate schema for saas_template_schema.
    """
    try:
        # Step 1: Check if schema already exists
        existing_schema = get_existing_schema(saas_edge_id, template_id)
        if existing_schema:
            print("Schema found in the database.")
            return existing_schema

        print("No schema found. Generating and saving schema...")
        # Step 2: Fetch template attributes and generate schema
        template_attributes = fetch_template_attributes(saas_edge_id, template_id)
        if not template_attributes:
            raise ValueError(
                "No attributes found for the given saas_edge_id and template_id."
            )

        generated_schema = generate_json_schema(template_attributes)

        # Step 3: Save the schema to the database
        insert_or_update_schema(saas_edge_id, template_id, generated_schema)
        return generated_schema

    except Exception as e:
        print(f"Error in main function: {e}")


genarate_template_schema(saas_edge_id, request_args.get("template_id"))

In [ ]:
# Usage example:
try:
    template_id = request_args.get("template_id")
    if not template_id:
        raise ValueError("template_id not found in request_args")
        
    template_data = fetch_channel_attr_template(saas_edge_id, template_id)
    template_details = fetch_saas_channel_templates(saas_edge_id, template_id)  
    if not template_data:
        print("WARNING: No template data retrieved or an error occurred during the fetch operation.")

except Exception as e:
    print("ERROR: An unexpected error occurred while fetching template data.")
    print(f"ERROR: {e}")
    print(f"ERROR: {traceback.format_exc()}")

In [ ]:
  # Return original value if conversion fails



filtered_template_data = {}

try:
    if template_data:
        # filtered_template_data = [
        #     {
        #         'column_name': item.get('column_name', ''),  # Default to empty string if not found
        #         'field_mapping': item.get('field_mapping', {})  # Default to empty dict if not found
        #     }
        #     for item in template_data
        #     if isinstance(item, dict)  # Ensure item is a dictionary
        # ]
        filtered_template_data = {
            item.get('column_name', ''): {
                "field_mapping":item.get('field_mapping', ""),
                "accepts_multiple_values":item.get('accepts_multiple_values', False),
                "transformation_function":item.get("transformation_rules", ""),
                "data_type":item.get("data_type", "text")
            } 
            for item in template_data
            
        }
        if not filtered_template_data:
            print("WARNING: No valid template data entries found after filtering")
    else:
        print("WARNING: template_data is None or empty")

    print("Filtered template data:", filtered_template_data)

except Exception as e:
    print(f"ERROR: Failed to process template data: {str(e)}")
    print(f"ERROR: {traceback.format_exc()}")
    filtered_template_data = {}  # Ensure we always return a list

In [ ]:
request_args

In [ ]:
file_url = request_args.get("file_url", {}).get("url")
create_new_product = request_args.get("feed_settings", {}).get("create_new_product", False)
print(file_url)

if template_details.get("meta",{}):
    saastify_table = template_details.get("meta",{}).get("saastify_table", "")
else:
    saastify_table = "products"
    # create_new_product = template_details.get("meta", {}).get("create_new_product", False)
print(saastify_table)

In [ ]:
 # saastify_table = "product_parent_variants"

In [ ]:
try:
    file_parse = FileParser()
    print(file_url)
    data_df = file_parse.load(file_url)
    properties_schema = list(data_df)
except Exception as e:
    print(f"ERROR: Failed to parse the file: {str(e)}")
    print(f"ERROR: {traceback.format_exc()}")
    properties_schema = []


In [ ]:
# print(properties_schema)
# print(filtered_template_data)
# print(template_data)

# replace|||value|||C:\OrderWise Data\||https://storage.googleapis.com/edge-assets/8dddba20-2928-45b2-a26f-381515f9ea68/catalog-edge/dam/public/|;|replace|||value||| ||%20


In [ ]:
# # TODO Support for multiple rules

# def apply_transformation(value, transformation_rules: str):
#     """
#     Apply multiple transformation rules to a value with comprehensive error handling.
#     Rules are separated by |;| character sequence.
#     """
#     try:
#         # Handle None/empty values
#         if value is None or transformation_rules is None:
#             return value
            
#         # Ensure transformation_rules is a string
#         if not isinstance(transformation_rules, str):
#             print(f"WARNING: Invalid transformation rule type: {type(transformation_rules)}")
#             return value
        
#         # Normalize backslashes in the input value if it's a string
#         if isinstance(value, str):
#             transformed_value = value.replace('\\', '/')
#         else:
#             transformed_value = value
            
#         # Split rules by |;| character sequence and process each rule in sequence
#         rules = [rule.strip() for rule in transformation_rules.split('|;|')]
#         for rule in rules:
#             # Basic string transformations
#             if rule == "uppercase":
#                 transformed_value = transformed_value.upper() if isinstance(transformed_value, str) else transformed_value
#             elif rule == "lowercase":
#                 transformed_value = transformed_value.lower() if isinstance(transformed_value, str) else transformed_value
#             elif rule == "strip":
#                 transformed_value = transformed_value.strip() if isinstance(transformed_value, str) else transformed_value
#             elif rule == "split_comma":
#                 if not isinstance(transformed_value, str):
#                     continue
#                 # Handle potential backslashes in comma-separated values
#                 transformed_value = [item.strip().replace('\\', '/') 
#                                    for item in transformed_value.split(",") 
#                                    if item.strip()]
                
#             elif rule.startswith("set|||"):
#                 try:
#                     parts = rule.split("|||")
#                     if len(parts) != 2:
#                         print(f"WARNING: Invalid set rule format: {rule}. Expected 2 parts.")
#                         continue
#                     _, value = parts
#                     transformed_value = value
#                 except Exception as e:
#                     print(f"WARNING: Error processing set rule: {rule}. Error: {str(e)}")
#                     continue
#             elif rule == "date_only":
#                 if isinstance(transformed_value, (pd.Timestamp, datetime, date)):
#                     transformed_value = transformed_value.strftime('%Y-%m-%d')  # Convert datetime to string with only date
#                 elif isinstance(transformed_value, str):
#                     try:
#                         # Handle cases where value is a string with datetime format
#                         parsed_date = pd.to_datetime(transformed_value, errors='coerce')
#                         if not pd.isnull(parsed_date):  
#                             transformed_value = parsed_date.strftime('%Y-%m-%d')
#                             print("new",transformed_value)
#                     except Exception as e:
#                         print(f"WARNING: Could not parse date for value '{transformed_value}'. Error: {str(e)}")
#             # Complex transformations
#             elif rule.startswith("replace|||"):
#                 if not isinstance(transformed_value, str):
#                     continue
#                 try:
#                     # Split using ||| delimiter
#                     parts = rule.split("|||")
#                     if len(parts) != 3:
#                         print(f"WARNING: Invalid replace rule format: {rule}. Expected 3 parts.")
#                         continue
                    
#                     _, _, replace_rule = parts
#                     original, new = replace_rule.split("||", 1)
                    
#                     original = original.strip().replace('\\', '/')
#                     new = new.strip().replace('\\', '/')
#                     original = original.strip().replace('\\', '/')
#                     new = new.strip().replace('\\', '/')
#                     transformed_value = transformed_value.replace(original, new)
                    
#                 except (IndexError, ValueError) as e:
#                     print(f"WARNING: Invalid replace rule format: {rule}. Error: {str(e)}")
#                     continue
                    
#             elif rule == "vlookup:map":
#                 if not isinstance(transformed_value, str):
#                     continue
#                 mapping = {
#                     "yes": True,
#                     "no": False,
#                     "true": True,
#                     "false": False,
#                     "1": True,
#                     "0": False,
#                     "y": True,
#                     "n": False
#                 }
#                 # Convert to lowercase and strip before mapping
#                 lookup_value = transformed_value.lower().strip()
#                 transformed_value = mapping.get(lookup_value, transformed_value)
            
#             # New transformation rule for adjusting negative numbers to zero
#             elif rule == "adjust_negative_to_zero":
#                 print("transformed_value",transformed_value)
#                 try:
#                     if isinstance(transformed_value, (int, float)):
#                         transformed_value = max(0, transformed_value)
#                     elif isinstance(transformed_value, list):
#                         transformed_value = [max(0, float(val)) if isinstance(val, (int, float, str)) else val 
#                                           for val in transformed_value]
#                     elif isinstance(transformed_value, str):
#                         try:
#                             num_value = float(transformed_value)
#                             transformed_value = str(max(0, num_value))
#                         except ValueError:
#                             print(f"WARNING: Could not convert string '{transformed_value}' to number for negative adjustment")
#                 except Exception as e:
#                     print(f"WARNING: Error applying negative adjustment: {str(e)}")
#                     continue
            
#             else:
#                 print(f"WARNING: Unknown transformation rule: {rule}")
#                 continue
        
#         return transformed_value
            
#     except Exception as e:
#         print(f"ERROR: Transformation failed for value '{value}' with rules '{transformation_rules}': {str(e)}")
#         return value  # Return original value on error

In [ ]:
import re
from datetime import datetime, date


# ─── Sentinel ───────────────────────────────────────────────────────────────
class RejectRow(Exception):
    pass

# ─── Transform helpers ──────────────────────────────────────────────────────
def uppercase(v, **kw): return v.upper() if isinstance(v, str) else v
def lowercase(v, **kw): return v.lower() if isinstance(v, str) else v
def strip(v, chars=None, **kw): return v.strip(chars) if isinstance(v, str) else v
def title_case(v, **kw): return v.title() if isinstance(v, str) else v
def capitalize(v, **kw): return v.capitalize() if isinstance(v, str) else v
def split_comma(v, **kw): return [x.strip() for x in v.split(',')] if isinstance(v, str) else v
def split(v, delimiter, **kw): return v.split(delimiter) if isinstance(v, str) else v
def join(v, delimiter, **kw):
    if isinstance(v, str): v = v.split()
    return delimiter.join(str(x) for x in v) if isinstance(v, (list, tuple)) else v
def replace(v, old, new, **kw): return v.replace(old, new) if isinstance(v, str) else v
def replace_regex(v, pattern, repl, **kw): return re.sub(pattern, repl, v) if isinstance(v, str) else v
def clean_numeric_value(v, **kw): return float(re.sub(r'[^\d.]', '', v)) if isinstance(v, str) else v
def clean_upc(v, **kw): return re.sub(r'[^\d]', '', v) if isinstance(v, str) else v
def clean_html(v, **kw): return re.sub(r'<.*?>', '', v) if isinstance(v, str) else v
def date_only(v, **kw):
    if isinstance(v, str):
        try: return datetime.strptime(v.strip(), "%Y-%m-%d %H:%M:%S").strftime("%Y-%m-%d")
        except ValueError: return v.split()[0] if " " in v else v
    if isinstance(v, (datetime, date)): return v.strftime("%Y-%m-%d")
    return v
def set_value(v, value, **kw): return value
def set_number(v, value, **kw): return int(value)
def zero_padding(v, value, **kw): return str(v).zfill(int(value))
def addition(v, amount, **kw): return float(v) + float(amount)
def subtraction(v, amount, **kw): return float(v) - float(amount)
def multiplication(v, factor, **kw): return float(v) * float(factor)
def division(v, divisor, **kw): return float(v) / float(divisor) if float(divisor) != 0 else None
def percentage(v, factor=100, **kw): return float(v) * float(factor)
def adjust_negative_to_zero(v, **kw):
    try: return max(0, float(v))
    except: return v
def vlookup_map(v, mapping=None, **kw):
    if isinstance(v, str) and mapping: return mapping.get(v.lower(), v)
    return v
def prefix(v, prefix_str="-", **kw):
    if isinstance(v, (list, tuple)): return [prefix_str + str(x) for x in v]
    if isinstance(v, str): return prefix_str + v
    return v
def suffix(v, suffix_str="_", **kw):
    if isinstance(v, (list, tuple)): return [str(x) + suffix_str for x in v]
    if isinstance(v, str): return v + suffix_str
    return v
def copy(v, **kw): return v
def rejects(v, **kw): raise RejectRow()

# ─── Registry ───────────────────────────────────────────────────────────────
TRANSFORMS = {
    "uppercase": uppercase, "lowercase": lowercase, "strip": strip, "title_case": title_case,
    "capitalize": capitalize, "split_comma": split_comma, "split": split, "join": join,
    "replace": replace, "replace_regex": replace_regex, "clean_numeric_value": clean_numeric_value,
    "clean_upc": clean_upc, "clean_html": clean_html, "date_only": date_only,
    "set": set_value, "set_number": set_number, "zero_padding": zero_padding,
    "addition": addition, "subtraction": subtraction, "multiplication": multiplication,
    "division": division, "percentage": percentage, "adjust_negative_to_zero": adjust_negative_to_zero,
    "vlookup_map": vlookup_map, "prefix": prefix, "suffix": suffix,
    "copy": copy, "rejects": rejects
}

# ─── Run one pipeline ───────────────────────────────────────────────────────
def apply_transformations(val, steps):
    v = val
    for st in steps:
        func = TRANSFORMS[st["name"]]
        try: v = func(v, **st.get("params", {}))
        except RejectRow: return None, True
    return v, False

# ─── DSL engine with broadcasting ───────────────────────────────────────────
def bulk_apply_pipe_rules(values_list, rule_strings):
    if not isinstance(values_list, list):
        raise ValueError("values_list must be a list")

    if isinstance(rule_strings, str):
        rule_strings = [rule_strings] * len(values_list)
    elif isinstance(rule_strings, list):
        if len(rule_strings) == 1 and len(values_list) > 1:
            rule_strings *= len(values_list)
        elif len(values_list) == 1 and len(rule_strings) > 1:
            values_list *= len(rule_strings)
        elif len(rule_strings) != len(values_list):
            raise ValueError("Length mismatch between values_list and rule_strings")
    else:
        raise ValueError("rule_strings must be str or list[str]")

    results = []
    for val, rule in zip(values_list, rule_strings):
        if not rule:
            results.append(val)
            continue

        steps = []
        for token_raw in rule.replace('+', '|;|').split('|;|'):
            token = token_raw.lstrip()
            if not token:
                continue

            # ---------- split ----------
            if token.startswith("split|"):
                raw = token[6:]
                if raw in ("|||", r"\|"):
                    delim = "|"
                elif raw.startswith("|||"):
                    delim = raw[3:] or "|"
                elif raw.startswith("||"):
                    tail = raw[2:]
                    delim = " " if tail in ("", " ") else tail
                elif raw.startswith(r"\|"):
                    delim = "|" + raw[2:]
                else:
                    delim = raw
                if delim != " ": delim = delim.strip()
                if delim == "": raise ValueError("split requires non-empty delimiter")
                steps.append({"name": "split", "params": {"delimiter": delim}})
                continue

            # ---------- join ----------
            if token.startswith("join|"):
                steps.append({"name": "join", "params": {"delimiter": token[5:]}})
                continue

            # ---------- prefix / suffix ----------
            if token.startswith("prefix|"):
                steps.append({"name": "prefix", "params": {"prefix_str": token[7:]}})
                continue
            if token.startswith("suffix|"):
                steps.append({"name": "suffix", "params": {"suffix_str": token[7:]}})
                continue

            # ---------- replace_regex ----------
            if token.startswith("replace_regex|"):
                pat, *rep = token[14:].split("||", 1)
                steps.append({"name": "replace_regex",
                              "params": {"pattern": pat, "repl": rep[0] if rep else ""}})
                continue

            # ---------- replace ----------
            if token.startswith("replace|"):
                try:
                    old, new = token[8:].split('|', 1)
                except ValueError:
                    raise ValueError(f"replace requires two parameters: {token}")
                steps.append({"name": "replace",
                              "params": {"old": old, "new": new}})
                continue

            # ---------- vlookup ----------
            if token.startswith("vlookup|"):
                mapping = {}
                for pair in token[8:].rstrip('|').split(','):
                    k, v = pair.split(':', 1)
                    v = v.strip()
                    mapping[k.strip().lower()] = \
                        True if v.lower() == "true" else \
                        False if v.lower() == "false" else \
                        float(v) if re.match(r'^-?\d+\.\d+$', v) else \
                        int(v) if v.isdigit() else v
                steps.append({"name": "vlookup_map", "params": {"mapping": mapping}})
                continue

            # ---------- arithmetic ----------
            for op in ("addition", "subtraction", "multiplication", "division", "percentage"):
                if token.startswith(f"{op}|"):
                    param = token.split('|', 1)[1]
                    key = ("amount" if op in ("addition", "subtraction") else
                           "factor" if op in ("multiplication", "percentage") else
                           "divisor")
                    steps.append({"name": op, "params": {key: param}})
                    break
            else:
                # ---------- strip with chars / set / zero_padding ----------
                if token.startswith("strip|"):
                    steps.append({"name": "strip",
                                  "params": {"chars": token.split('|',1)[1]}})
                elif token.startswith("set|"):
                    steps.append({"name": "set",
                                  "params": {"value": token.split('|',1)[1]}})
                elif token.startswith("set_number|"):
                    steps.append({"name": "set_number",
                                  "params": {"value": token.split('|',1)[1]}})
                elif token.startswith("zero_padding|"):
                    steps.append({"name": "zero_padding",
                                  "params": {"value": token.split('|',1)[1]}})
                elif token == "rejects":
                    steps.append({"name": "rejects"})
                else:
                    steps.append({"name": token})

        out, _ = apply_transformations(val, steps)
        results.append(out)

    return results

In [ ]:
import re

def apply_transformation(value, transformation_rules: str):
    """
    Adapter that preserves your old call site but normalizes legacy tokens:
    - 'set|||VALUE'        -> 'set|VALUE'
    - 'replace|||OLD||NEW' -> 'replace|OLD|NEW'   (optional but handy)
    Then forwards to bulk_apply_pipe_rules unchanged.
    """
    try:
        if transformation_rules is None:
            return value
        if not isinstance(transformation_rules, str):
            print(f"WARNING: Invalid transformation rule type: {type(transformation_rules)}")
            return value

        # Optional: keep your backslash → slash normalization
        if isinstance(value, str):
            value = value.replace("\\", "/")

        # --- Normalize legacy rule formats (minimal, safe) ---
        fixed_rules = transformation_rules

        # set|||VALUE  -> set|VALUE
        fixed_rules = re.sub(r'(?<!\w)set\|\|\|', 'set|', fixed_rules)

        # replace|||OLD||NEW -> replace|OLD|NEW
        # (only converts the first '||' after 'replace|||' to keep the rest intact)
        fixed_rules = re.sub(r'(?<!\w)replace\|\|\|([^|]+)\|\|', r'replace|\1|', fixed_rules)

        # Hand off directly to your DSL (no extra splitting here)
        return bulk_apply_pipe_rules([value], fixed_rules)[0]

    except Exception as e:
        print(f"ERROR: Transformation failed for value '{value}' with rules '{transformation_rules}': {str(e)}")
        return value


In [ ]:
def convert_value(value, data_type):
    import datetime

    """
    Converts the given value to the specified data type.
    If the value is already correct, it remains unchanged.
    """
    try:
        if data_type == "text":
            return str(value) if value is not None else ""

        elif data_type == "integer":
            return int(value) if isinstance(value, (int, float, str)) and str(value).isdigit() else 0

        elif data_type == "float":
            return float(value) if isinstance(value, (int, float, str)) and str(value).replace('.', '', 1).isdigit() else 0.0

        elif data_type == "boolean":
            return str(value).lower() in ["true", "1", "yes"]

        elif data_type == "datetime":
            if isinstance(value, (datetime.datetime, datetime.date, pd.Timestamp)):
                return value.strftime('%Y-%m-%d %H:%M:%S')  # Convert datetime to string
            try: 
                return str(pd.to_datetime(value))  # Convert if possible
            except:
                return str(value)
        elif data_type == "timestamp":
            if isinstance(value, (datetime.datetime, datetime.date, pd.Timestamp)):
                return value.strftime('%Y-%m-%d %H:%M:%S')  # Convert datetime to string
            try: 
                return str(pd.to_datetime(value))  # Convert if possible
            except:
                return str(value)
        
                
        elif data_type == "date":
            if isinstance(value, (datetime.datetime, datetime.date, pd.Timestamp)):
                return value.strftime('%Y-%m-%d ')  # Convert timestamp to YYYY-MM-DD (No time)
            try:
                return pd.to_datetime(value).strftime('%Y-%m-%d')  # Convert if possible
            except:
                return str(value)

        elif data_type == "jsonb":
            if isinstance(value, str):
                try:
                    return json.loads(value)  # Convert JSON string to dict
                except json.JSONDecodeError:
                    return {}  # Return empty dict if invalid JSON
            return value  # If already dict, return as is

        elif data_type == "array":
            return value if isinstance(value, list) else [value]  # Ensure it's always a list

        else:
            return value  # Keep as is for unknown data types

    except Exception as e:
        print(f"WARNING: Failed to convert value '{value}' to {data_type}: {e}")
        return value




def build_request_objects(data_df, filtered_template_data):
    request_objects = []
    
    
    try:
        for index, row in data_df.iterrows():
            
            
            request_obj = {}
            # Convert row to dictionary and handle NaN values
            row_dict = {
                k: (None if pd.isna(v) else v) 
                for k, v in row.to_dict().items()
            }
            
            for col_name in data_df.columns:
                
                if col_name not in filtered_template_data:
                    continue  # column not in template, skip

                value = row_dict.get(col_name)
                # if value is None or (isinstance(value, str) and value.strip() == ""):
                #     continue  # value is empty, skip

                field_config = filtered_template_data[col_name]
                if not field_config or not field_config.get("field_mapping"):
                    continue
                try:
                    if pd.notnull(value):
                        # Only clean if it's numeric (int/float) and ends in .0
                        if isinstance(value, (int, float)) and float(value).is_integer():
                            value = str(int(value))  # 123456.0 → "123456"
                        else:
                            value = str(value).strip()  # Keep as-is for strings like "123456.0lkw"
                except Exception as e:
                    print(f"WARNING: Error cleaning value '{value}': {e}")

                if value is None:
                    value = None
                
                field_mapping = field_config['field_mapping']
                accepts_multiple = field_config.get('accepts_multiple_values', False)
                transformation_function = field_config.get("transformation_function", "")
                data_type = field_config.get("data_type", "")
                

                if data_type:
                    value = convert_value(value, data_type)
                   
                # trasnformation needs to be applied to value
                if transformation_function:
                    value = apply_transformation(value, transformation_function)
                    # while isinstance(temp_value, list) and temp_value:
                    #     value = temp_value[0]
                
                # Handle nested mappings (e.g., "raw_product_data.field_name")
                if "." in field_mapping:
                    parent_key, child_key = field_mapping.split(".", 1)
                    
                    # Initialize parent dictionary if it doesn't exist
                    if parent_key not in request_obj:
                        request_obj[parent_key] = {}
                        
                    request_obj[parent_key][child_key] = value.strip() if isinstance(value, str) else value
                else:
                    # Handle non-nested fields
                    if accepts_multiple and isinstance(value, str):
                        request_obj[field_mapping] = [v.strip() for v in value.split(',') if v.strip()]
                    else:
                        request_obj[field_mapping] = value.strip() if isinstance(value, str) else value
            
            request_objects.append(request_obj)
            
        print(f"INFO: Successfully created {len(request_objects)} request objects")
        return request_objects
        
    except Exception as e:
        print(f"ERROR: Failed to build request objects: {str(e)}")
        print(f"ERROR: {traceback.format_exc()}")
        return []

In [ ]:
request_objects = build_request_objects(data_df, filtered_template_data)
# print(request_objects)
# print(data_df)
# print(filtered_template_data)


In [ ]:
# Update job details
def update_job_details(job_id, total_count,success_count, failure_count, failed_url=None,failed_report=None):
    """
    Update job details in the database with success/failure counts and failed job summary link
    """
    try:
        conn = get_db_connection()
        cursor = conn.cursor()

        job_response = {
            "total": total_count,
            "success": success_count,
            "failed": failure_count,
        }

        update_fields = ["job_response = %s"]
        update_values = [json.dumps(job_response)]
        

        # Add failed_job_summary_link if present
        if failed_url:
            update_fields.append("failed_job_summary_link = %s")
            update_values.append(failed_url)

        # Add job_id for WHERE clause
        update_values.append(job_id)
        print("update_values",update_values)

        query = f"""
        UPDATE saas_edge_jobs
        SET {", ".join(update_fields)}
        WHERE request_id = %s
        RETURNING *;
        """
        
        cursor.execute(query, update_values)
        updated_row = cursor.fetchone()
        conn.commit()

        print(f"INFO: Successfully updated job details for job_id: {job_id}")
        return updated_row

    except Exception as e:
        if conn:
            conn.rollback()
        print(f"ERROR: Failed to update job details: {str(e)}")
        print(f"ERROR: {traceback.format_exc()}")
        return None
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()


In [ ]:


# Upload files to gcp
def sanitize_path(path):
    """
    Sanitize path by replacing forward slashes with underscores
    """
    return path.replace('/', '_').replace(".ipynb","").strip('')

def get_output_filename(saas_edge_id, job_path,file_type):
    """
    Generate standardized output filename with proper date formatting
    """
    try:
        current_date = datetime.now()
        date_str = current_date.strftime('%Y-%m-%d')
        timestamp_str = current_date.strftime('%H%M%S')
        sanitized_job_path = sanitize_path(job_path)
        
        return f"{saas_edge_id}/catalog-edge/job-reports/{sanitized_job_path}/{date_str}/import-failed-list-{timestamp_str}.{file_type}"
    except Exception as e:
        print(f"Error generating output filename: {str(e)}")
        # Fallback filename if something goes wrong
        return f"{saas_edge_id}/catalog-edge/job-reports/error_report.json"

def upload_to_gcp_bucket(file_data, file_name, bucket_name, base_path="", credentials=None):
    """
    Generic function to upload data to GCP bucket using existing credentials
    """
    try:
        # Create credentials object from the service account info
        gcp_credentials = service_account.Credentials.from_service_account_info(credentials)
        storage_client = storage.Client(credentials=gcp_credentials)
        
        bucket = storage_client.bucket(bucket_name)
        
        # Combine base path with file name
        full_path = f"{base_path.rstrip('/')}/{file_name}" if base_path else file_name
        blob = bucket.blob(full_path)
        
        # Add retry logic for upload
        retry_count = 0
        max_retries = 3
        
        while retry_count < max_retries:
            try:
                # Convert data to appropriate format for upload
                if isinstance(file_data, (list, dict)):
                    # Convert to JSON string for lists and dictionaries
                    upload_data = json.dumps(file_data, indent=2)
                    blob.upload_from_string(upload_data, content_type='application/json')
                elif isinstance(file_data, str):
                    blob.upload_from_string(file_data)
                elif isinstance(file_data, bytes):
                    blob.upload_from_string(file_data, content_type='application/octet-stream')
                else:
                    # For file-like objects
                    blob.upload_from_file(file_data)
                    
                # Generate public URL
                url = f"https://storage.googleapis.com/{bucket_name}/{full_path}"
                print(f"Successfully uploaded {file_name} to {url}")
                return True, url
                
            except Exception as upload_error:
                retry_count += 1
                if retry_count == max_retries:
                    print(f"Failed to upload {file_name} after {max_retries} attempts: {str(upload_error)}")
                    return False, ""
                print(f"Retry {retry_count} for {file_name}")
                
    except Exception as e:
        print(f"Error uploading to GCP bucket: {str(e)}")
        print(f"Traceback: {traceback.format_exc()}")
        return False, ""

In [ ]:
# GCP credentials should be loaded from GOOGLE_APPLICATION_CREDENTIALS environment variable
# or use Application Default Credentials
# Example: os.getenv("GOOGLE_APPLICATION_CREDENTIALS")


In [ ]:
process_data_import(saas_edge_id, saastify_table, request_objects, create_new_product=create_new_product)

In [ ]:
 # def convert_to_us_central(date_str):
 #        """Convert given date string to US Central Time (assuming it's in ISO 8601 or UTC)."""
 #        if not date_str:
 #            return None
 #        try:
 #            # Parse the string to a datetime object
 #            dt = datetime.fromisoformat(date_str.replace("Z", "+00:00"))
 #            # Convert to US Central Time
 #            central_tz = pytz.timezone("US/Central")
 #            dt_central = dt.astimezone(central_tz)
 #            # Return in ISO format without microseconds
 #            return dt_central.strftime("%Y-%m-%d %H:%M:%S")
 #        except Exception as e:
 #            print(f"Date conversion error for {date_str}: {e}")
 #            return date_str  

In [ ]:
# convert_to_us_central("2025-09-04")